In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import gc
import os
import math
import copy
import types
import yaml
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import (
    CrossEntropyLoss,
    MSELoss
)
from torch.utils.data import DataLoader

import evaluate

import transformers
from transformers import (
    AutoModelForTokenClassification,
    AutoConfig,
    T5EncoderModel,
    T5Tokenizer,
    T5PreTrainedModel,
    T5ForConditionalGeneration,
    pipeline,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    set_seed,
    EvalPrediction,
    )
from transformers.modeling_outputs import TokenClassifierOutput

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    get_peft_config,
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training
    )

from datasets import Dataset

import src.config as config
import src.config
import src.data
import src.model_new

from src.model_working import (
    get_prottrans_tokenizer_model,
    df_to_dataset,
    inject_linear_layer,
    )
from src.utils import get_project_root_path

/Users/finnlueth/Developer/gits/prottrans-t5-signalpeptide-prediction/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


---
## Setup and Variables

In [3]:
base_model_name = config.base_model_name
print("Base Model:\t", base_model_name)
print("MPS:\t\t", torch.backends.mps.is_available())
ROOT = get_project_root_path()
print("Path:\t\t", ROOT)
device = torch.device('cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f"Using device:\t {device}")

Base Model:	 Rostlab/prot_t5_xl_uniref50
MPS:		 True
Path:		 /Users/finnlueth/Developer/gits/prottrans-t5-signalpeptide-prediction
Using device:	 mps


---
## Create Tokenizer and Load Model

In [4]:
model_architecture = T5EncoderModel
t5_tokenizer, t5_base_model = get_prottrans_tokenizer_model(base_model_name, model_architecture)

---
## Load Data, Split into Dataset, and Tokenize Sequences

In [5]:
FASTA_FILENAME = '5_SignalP_5.0_Training_set.fasta'
annotations_name = 'Label' # Choose Type or Label

df_data = src.data.process(src.data.parse_file(ROOT + '/data/raw/' + FASTA_FILENAME))

dataset_signalp = src.model_new.create_datasets(
    splits=src.config.splits,
    tokenizer=t5_tokenizer,
    data=df_data,
    annotations_name=annotations_name,
    dataset_size=src.config.dataset_size,
    encoder=src.config.select_encodings[annotations_name],
    )

del df_data

/Users/finnlueth/Developer/gits/prottrans-t5-signalpeptide-prediction/.venv/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/Users/finnlueth/Developer/gits/prottrans-t5-signalpeptide-prediction/.venv/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


---
## Apply LoRA

In [6]:
lora_config = LoraConfig(
        # task_type=TaskType.TOKEN_CLS,
        inference_mode=False,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=['q', 'k', 'v', 'o'],
        # target_modules=['o'],
        bias="none",
    )
t5_lora_model = get_peft_model(t5_base_model, lora_config)
t5_lora_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,212,073,984 || trainable%: 0.32441584027926795


---
## Training Loop
https://huggingface.co/docs/peft/task_guides/token-classification-lora

In [7]:
t5_lora_model = inject_linear_layer(
    t5_lora_model=t5_lora_model,
    num_labels=config.label_decoding.__len__(),
    dropout_rate=config.dropout_rate
    )

In [8]:
# seqeval_metric = evaluate.load("seqeval")
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
# roc_auc_score_metric = evaluate.load("roc_auc", "multiclass")
# roc_auc_score = evaluate.load("roc_auc")
matthews_correlation_metric = evaluate.load("matthews_correlation")

def batch_eval_elementwise(predictions: np.ndarray, references: np.ndarray):
    results = {}
    # predictions = np.nan_to_num(predictions).argmax(axis=-1)
    predictions = predictions.argmax(axis=-1)
    
    results.update({'accuracy_metric': np.average([accuracy_metric.compute(predictions=x, references=y)['accuracy'] for x, y in zip(predictions, references)])})
    results.update({'precision_metric': np.average([precision_metric.compute(predictions=x, references=y, average='micro')['precision'] for x, y in zip(predictions, references)])})
    results.update({'recall_metric': np.average([recall_metric.compute(predictions=x, references=y, average='micro')['recall'] for x, y in zip(predictions, references)])})
    results.update({'f1_metric': np.average([f1_metric.compute(predictions=x, references=y, average='micro')['f1'] for x, y in zip(predictions, references)])})
    # results.update({'roc_auc': np.average([roc_auc_score_metric.compute(prediction_scores=x, references=y, average='micro')['roc_auc'] for x, y in zip(predictions, references)])})
    results.update({'matthews_correlation': np.average([matthews_correlation_metric.compute(predictions=x, references=y, average='micro')['matthews_correlation'] for x, y in zip(predictions, references)])})
    return results
# display(batch_eval_elementwise(predictions.numpy(), references.numpy()))

def compute_metrics(p):
    # print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= preds compute_metrics start =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    predictions, references = p
    results = batch_eval_elementwise(predictions=predictions, references=references)
    # print(results)
    # print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= preds compute_metrics stop =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    return results
# metrics = compute_metrics((predictions, references))

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer=t5_tokenizer)

training_args = TrainingArguments(
    output_dir='./checkpoints',
    learning_rate=config.lr,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.num_epochs,
    logging_steps=config.logging_steps,
    # save_strategy="steps",
    # save_steps=config.save_steps,
    evaluation_strategy="steps",
    eval_steps=1,
    # gradient_accumulation_steps=accum,
    # load_best_model_at_end=True,
    # save_total_limit=5,
    seed=42,
    # fp16=True,
    # deepspeed=deepspeed_config,
    remove_unused_columns=False,
    label_names=['labels'],
)

trainer = Trainer(
    model=t5_lora_model,
    args=training_args,
    train_dataset=dataset_signalp['train'],
    eval_dataset=dataset_signalp['valid'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [10]:
# print(next(t5_lora_model.parameters()).is_cuda)
# print(t5_lora_model.device)
# print(config.label_decoding)

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [12]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

self.get_base_model().forward <bound method T5EncoderModel.forward of T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(
                in_features=1024, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): Linear(
                in_features=1

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.790602684020996, 'eval_accuracy_metric': 0.14084507042253522, 'eval_precision_metric': 0.14084507042253522, 'eval_recall_metric': 0.14084507042253522, 'eval_f1_metric': 0.14084507042253522, 'eval_matthews_correlation': 0.013285066517684194, 'eval_runtime': 19.0725, 'eval_samples_per_second': 0.157, 'eval_steps_per_second': 0.052, 'epoch': 1.0}
{'train_runtime': 72.3093, 'train_samples_per_second': 0.124, 'train_steps_per_second': 0.014, 'train_loss': 1.7924234867095947, 'epoch': 1.0}


TrainOutput(global_step=1, training_loss=1.7924234867095947, metrics={'train_runtime': 72.3093, 'train_samples_per_second': 0.124, 'train_steps_per_second': 0.014, 'train_loss': 1.7924234867095947, 'epoch': 1.0})

In [ ]:
# metrics=trainer.evaluate()
# print(metrics)

In [ ]:
result_log = pd.DataFrame(trainer.state.log_history)
display(result_log)

---

In [ ]:
adapter_location = '/models/testing_2'
t5_lora_model.save_pretrained(ROOT + adapter_location)

---

In [ ]:
def predict_model(sequence: str, tokenizer: T5Tokenizer, model):
    # print('sequence', sequence)
    tokenized_string = tokenizer.encode(sequence, padding=True, truncation=True, return_tensors="pt", max_length=1024)
    # print('tokenized_string', tokenized_string)
    with torch.no_grad():
        output = model(tokenized_string.to(device))
    # print('output', output)
    return output

def tranlate_logits(logits):
    return [src.config.label_decoding[x] for x in logits.argmax(-1).tolist()[0]]

In [ ]:
_ds_index = 2
_ds_type = 'test'

_inids_test = t5_tokenizer.decode(dataset_signalp[_ds_type][_ds_index]['input_ids'])
_labels_test = dataset_signalp[_ds_type][_ds_index]['labels']
_labels_test_decoded = [src.config.label_decoding[x] for x in _labels_test]
print(_inids_test)
print(_labels_test)
print(_labels_test_decoded)

In [ ]:
preds = predict_model(_inids_test, t5_tokenizer, t5_lora_model)

In [ ]:
_res = tranlate_logits(preds.logits.cpu().numpy())
print(_res)